# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: This is a classification problem. This is because a pass/fail label is a discrete featureset, rather than a continuous one. We are not modelling continuous data which would be more suited to regression.
A more advanced problem seeking to improve a student's grades might instead be a regression problem.
**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = student_data[student_data.passed=="yes"].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data.passed=="no"].shape[0]

# TODO: Calculate graduation rate
grad_rate = (n_passed/n_students)*100

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ( "Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  higher internet  romantic  famrel  freetime goout Dalc Walc health absences  
0    yes       no        no       4         3     4    1    1     

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ( "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train ,X_test ,y_train ,y_test = train_test_split(X_all , y_all , test_size = num_test, train_size = num_train , random_state = 0)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: All models have been selected due to their inherent support for Classification type relationships.  Three models have been selected - Naive Bayes, Decision Trees and Support Vector Machines with some key information on each<sup>[1][2]</sup>:
<h3>Naive Bayes (GaussianNB)</h3>
- This model has been used for medical and biological applications, for example to predict DNA-binding proteins to help advance our  understanding of protein functions.<sup>[3]</sup>  Comparing other machine learning frameworks, the model performed best due to its abililty to handle lots of features.

- **Strengths:**  Naive Bayes performs well when inputs are independent from one another, and when there are many features
    - Few parameters so it's hard to overfit
    - Fast training speed
    - Good at seperating signal from noise
    - Able to handle many features
- **Weaknesses:** Naive Bayes performs badly for problems where probabilities are sometimes greater than zero for a particular classification.
    - Simplistic nature can yield bad generalizations or suboptimal solutions
    - Doesn't account for interactions between features
    - There is a high bias when there is a small amount of data
- **Applicability:** Our student data has a lot of features - 30 in total - which could influence the pass rate.  Naive Bayes could be a good candidate on this basis.  Assuming a typical school environment where high-end hardware is not available for generating the model, this will be a good candidate for lightweight learning.

<h3>Decision Trees</h3>
- This model has been used for Speech Recognition technologies, and recently applied to develop models for accented speech<sup>[4]</sup>.  By optimizing the tree and/or pruning it, algorithms have been improved for speech-recognition applications.  A key advantage of Decision Trees for accented speech was the favourable performance for adaptive data techniques - that is, using data from other models and implementing them into a current model.  

- **Strengths:** Decision trees perform well with certain boolean functions, and when the model can be built with smaller decision trees rather than bigger trees.
    - Fast training speed
    - Easy visualization
- **Weaknesses:** Decision trees perform badly if the tree grows quickly, and can overfit 
    - Trees can grow very fast
    - High memory footprint
    - Overfitting happens very easily
    - Generalize relatively poorly
- **Applicability:** Considering we have a large number of "yes"/"no" (i.e boolean) features in our dataset, this is well suited to a decision tree.  Moreover, we only have 395 data points, there is not a high risk of intense memory and CPU usage.  Even if the student base increases, it would likely stay in the same order of magnitude.  It is also easy to visualize whcih would be great for intepretation by the school board.

<h3>Support Vector Machines<sup>[5]</sup></h3>
- This model has been applied to the stock market to forecast movement of different markets, and found to work well in markets with either high volatility or lower volatility<sup>[6]</sup>.  In this case, Support Vector Machines were chosen because they tend to outperform other methods for financial time-series forecasting.

- **Strengths:** Support Vector Machines perform well when there is a definite distinction between two classes
    - Fast training speed
    - Good for feature-intense data, e.g. text
- **Weaknesses:** Support Vector Machines perform badly when the classification problem is not binary
    - Doesn't take into account all data equally, so can't see overall trends in data
    - A good choice in kernel is required, which is not always clear
    - Slow in test phase
- **Applicability:** Considering we have a binary classification - pass or fail - Support Vector Machines become a good candidate as a model.  Once again, assuming a typical school environment where high-end hardware is not available for generating the model, this will be a good candidate for relatively lightweight learning.

### K Nearest Neighbors

##### **Complexity**

Space will be consumed faster than runtime for training.  KNN defers computation to when you try to use it for prediction, so it's training time is O(1).  Prediction runtime, however, will go up as we add data. KNN needs to find the K closest points to a new data point, which means examining every point in the dataset (barring some clever sectioning of the data).  If you add 3 more data points to the set, prediction will have to calculate 3 more distances.  This seems to be O(n).

Space should be O(n) as for each additional data point, we consume 1 unit of additional space to store it for use during classification of novel data.


#####  **General Application**

KNN has the advantage of being a fairly simple algorithm to intuit the behavior of.  It just finds the K most similar points to a new data point, and uses a vote betwen them to decide how to classify it.  This works well for low dimensionality space that is relatively well clustered.  One downside of the algorithm is that for clusters that have complex decision boundaries, the closest points might not be a good measure of where the new point should fit.  Another drawback is that the prediction function is computationally intensive for large data sets, so adding more data will help the accuracy, but hinder the runtime for each prediction.

##### **Inclusion in Experiment**

Nearest Neighbors seems to have some intuitively correct application to this problem. If a student's habits, health, and employment are all very close to points for other passing students, it seems likely that student will be more likely to pass.**

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ( "Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state=42)
clf_C = SVC(random_state=29)
clf_D = KNeighborsClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size

#Naive_Bayes
print("\n Naive Bayes for 100 : \n ")
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print("\n Naive Bayes for 200 : \n ")
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print("\n Naive Bayes for 300 : \n ")
train_predict(clf_A , X_train_300, y_train_300, X_test, y_test)

print("--------------------------------------------------------------------")
#Decision Trees
print("\n Decision Trees for 100 : \n ")
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print("\n Decision Trees for 200 : \n ")
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print("\n Decision Trees for 300 : \n ")
train_predict(clf_B , X_train_300, y_train_300, X_test, y_test)

print("--------------------------------------------------------------------")

#SVM
print("\n SVM for 100 : \n ")
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print("\n SVM for 200 : \n ")
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print("\n SVM for 300 : \n ")
train_predict(clf_C , X_train_300, y_train_300, X_test, y_test)

print("--------------------------------------------------------------------")

#KNN
print("\n KNN for 100 : \n ")
train_predict(clf_D, X_train_100, y_train_100, X_test, y_test)
print("\n KNN for 200 : \n ")
train_predict(clf_D, X_train_200, y_train_200, X_test, y_test)
print("\n KNN for 300 : \n ")
train_predict(clf_D , X_train_300, y_train_300, X_test, y_test)



 Naive Bayes for 100 : 
 
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0018 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8550.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7481.

 Naive Bayes for 200 : 
 
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0016 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 0.8321.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7132.

 Naive Bayes for 300 : 
 
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0026 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8088.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7500.
--------------------------------------------------------------------

 Decision Trees for 100 : 
 
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0205 seconds
Made predictions in 0.0007

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Naive-Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.002                | 0.0009            |      0.8550         |     0.7481            |
| 200               |        0.0015          |        0.0008                |        0.8321          |             0.7132    |
| 300               |           0.0016              |      0.0013                  |     0.8088            |    0.7500      |

** Classifer 2 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0019                 |   0.0007                     |   1.0000               |     0.7258            |
| 200               |     0.0025             |      0.0008                  |          1.0000        |        0.7385         |
| 300               |         0.0042                |           0.0011        |        1.0000    |     0.7304|

** Classifer 3 -SVM  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0038            |     0.0018               |    0.8599              |   0.7838              |
| 200               |             0.0064            |       0.0031                 |     0.8693             |    0.7755             |
| 300               |              0.0133           |       0.0032                 |       0.8692           |     0.7586            |


** Classifer 3 -KNN 

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0020        |     0.0047      |    0.7972              |   0.7068              |
| 200               |             0.0017            |     0.0065                   |           0.8571       |     0.7121            |
| 300               |            0.0017             |      0.0080                  |     0.8722             |  0.7482               |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: 
In the comparison process I evaluated 4 common classifier algorithms (KNN, Decision Tree, Naive Bayes, and SVM) over the sample dataset.
Generally speaking, all training times are in the same order of magnitude, with Support Vector Machines being the highest.  In terms of Prediction times, Support Vector Machines was also generally the highest.

Despite this, the maximal time is still less than a millisecond and is unlikely to cause problems even for schools with modest-hardware.  Hence, it is reccomended to choose the Support Vector Machines as a model. It has the most consistent performance regardless of training set size, and consistently delivers high F<sub>1</sub> scores around 0.75-0.78.  On the other hand, Decision Tree delivers lower, less robust performance from 0.72-0.73 and GaussianNB delivers a score of around 0.71-0.75.  Clearly, Support Vector Machines have the most robust and strongest model going forward.

Future training set sizes are unlikely to increase by an order of magnitude so the prediction and training times are unlikely to increase drastically.  Moreover, even with a large training and prediction time, it is likely that the model would be trained and used on occasion at the end of each academic year rather than persistently.  This is because student performance and new school strategies are likely to be evaluated thoroughly once per year.  Considering this, performance and time are unlikely to be mission-critical even if order of magnitudes higher (i.e. seconds).  This means it is fine to stick with Support Vector Machines despite any performance concerns.**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: > 
>Any description of a technical topic to a non-technical audience is likely to be fraught with poor assumptions about the transparency of given jargon and the struggle to find the appropriate level of abstraction to gain the necessary intuition for a given solution.  With this in mind I'll try to write simply to minimize poor assumptions on my part about any particular piece of foundational knowledge: The model selected from the experiment so far is known as a Support Vector Machine Classifier.  The name does accurately represent some of the underlying mathematical concepts at play, but rather than dive too deeply into the derivation of the name instead let's focus on how it works.

> SVMs can be thought of as an attempt to plot all the data we have on a grid, and draw the line between the categories that is as close as possible to being evenly drawn between them (maximizing the margin).  Imagine taking each student and assigning them a shirt color (red if they failed, green if they passed) and having them all stand on a football field at points are calculated as a function of all of the things we know about them (so the same employment, habits, demographic would place a student at the same point on the field).  Now we start trying to set out traffic cones on the field to divide the students in red shirts from the students in green shirts, trying to keep the cones as far away from any individual student as possible.  Now if we took any new student, not knowing yet whether they would pass or fail, we could still find the right spot on the field for them by asking about their habits, demographic, and employment.  Then we just look at where the new student is standing, whether that point is on a red side of the divison, or the green side.  That's how we make our prediction.  It's a bit of a funny picture to imagine, but it's a decent thought experiment to help set out how SVMs work in a familiar setting.




### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [13]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

def f1_wrap(y_true, y_predict):
    return f1_score(y_true, y_predict, pos_label='yes')

# TODO: Create the parameters list you wish to tune
parameters = {'C':range(1,6),'kernel':['linear','poly','rbf','sigmoid'],'degree':range(1,6)}

# TODO: Initialize the classifier
clf = SVC(random_state = 0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_wrap)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print("\nBest Score :{}\n".format(grid_obj.best_score_))

print("\nBest params :{}\n".format(grid_obj.best_params_))


      

# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The defaul


Best Score :0.8266917223826667


Best params :{'C': 2, 'degree': 1, 'kernel': 'poly'}

Made predictions in 0.0047 seconds.
Tuned model has a training F1 score of 0.8351.
Made predictions in 0.0018 seconds.
Tuned model has a testing F1 score of 0.8000.


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packa

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: - 
- Final F<sub>1</sub> score for training: 0.8351
- Final F<sub>1</sub> score for testing: 0.8000
- Previous F<sub>1</sub> score for training: 0.8692
- Previous F<sub>1</sub> score for testing: 0.7586
- **Delta F<sub>1</sub> score for training: -0.0341**
- **Delta F<sub>1</sub> score for testing: +0.0414**

The training F<sub>1</sub> has actually *decreased* after optimazation.  This is unsurprising, and is due to the fact that the previous model would have been overfitted towards the training data.  This time, the training parameters have been cross-validated in order to make the training and testing scores more consistent.  Hence, although the training score has apparently decreased, it is actually less over-fitted and more consistent with testing.

The testing F<sub>1</sub> score has increased notably after optimization, now bringing the classification into the 80%s - a 0.41% gain.  This is a notable optimization, and certainly worth the minimal effort to tune the parameters.  However, it has not massively increased the accuracy which may be an indicator of the limitation of Support Vector Machines themselves - a different, more intensive model, might increase the accuracy further.

Overall, it forms a good predictive model for students in class and what factors contribute towards their pass and failure. **

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.